# Bayes Optimization

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
%matplotlib inline

## Objective Function

In [2]:
def target_function(x):
    l2 = np.linalg.norm(x)
    return np.sin(l2/2)
def estimate(x):
    noise = np.random.normal(loc=0.0, scale=0.0001)
    return target_function(x) + noise


In [3]:
def kernel_function(x_1, x_2, theta_f, l):
    diff = [x_1[i]-x_2[i] for i in range(len(x_1))]
    l2 = np.linalg.norm(diff)**2
    return theta_f**2*np.exp(-l2/2/l**2)

## Update Prior Distibution

In [4]:
def covar_matrix(x,theta_f, l):
    n = len(x)
    matrix = np.ones((n,n))
    for i in range(n):
        for j in range(i,n):
            cor = kernel_function(x[i],x[j], theta_f, l)
            matrix[i][j] = matrix[j][i] = cor
    return matrix

In [5]:
# suppose we alredy knew x, y
def prior_mle(parameter, x, y):
    theta_f,l = parameter[0], parameter[1]
    theta_y = 0.0001
    n = len(x)
    matrix = covar_matrix(x, theta_f,l) + theta_y*np.eye(n)
    inverse = np.linalg.inv(matrix)
    np_x = np.asarray(x)
    np_y = np.asarray(y)
    mle = -np.matmul(np.matmul(np_y, inverse), np_y) - np.log(np.linalg.det(matrix))
    return -mle
    
    

## Posterior mean and variance

In [6]:
def posterior_dist(new_point, parameter, x, y):
    theta_f,l = parameter[0], parameter[1]
    theta_y = 0.0001
    n = len(x)
    #distance between x_1, x_2 ... and new_point
    d = []
    for i in range(n):
        d.append(kernel_function(new_point,x[i],theta_f, l))
    var = kernel_function(new_point,new_point,theta_f, l)
    matrix = covar_matrix(x, theta_f,l) + theta_y*np.eye(n)
    inverse = np.linalg.inv(matrix)
    np_d = np.asarray(d)
    np_x = np.asarray(x)
    np_y = np.asarray(y)
    posterior_mean = np.matmul(np.matmul(np_d, inverse), np_y)
    posterior_var = var-np.matmul(np.matmul(np_d, inverse), np_d)
    return posterior_mean, posterior_var

## Upper Confidence Bound


In [7]:
def ucb(new_point, parameter, x,y, beta):
    mean, var = posterior_dist(new_point, parameter, x, y)
    return -(mean + np.sqrt(beta)*np.sqrt(var))


## Bayes Optimization

In [8]:
x = [np.random.normal(loc=0.0, scale=0.01,size=2)]
y = [estimate(x[i]) for i in range(len(x))]

In [ ]:
bnds = ((-5, 5), (-5, 5))
for i in range(50):
    parameter = minimize(prior_mle,x0=(1,1),method='L-BFGS-B',args= (x,y)).x
    new_point = minimize(ucb,x0=(1,1), method='L-BFGS-B',bounds=bnds,args = (parameter, x,y, 1)).x
    x.append(new_point)
    y.append(estimate(new_point))

In [ ]:
plt.plot([i for i in range(len(y))], y)
plt.ylabel('Objective value')
plt.xlabel('Iteration');
